Old notebook, most has been moved/is obsolete.  
Still has some code for the diverse beam search experiments.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import importlib
import os
import sys

import torch
import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer, PreTrainedModel

from tqdm.auto import tqdm

from nltk.translate.bleu_score import sentence_bleu

module_path = os.path.abspath(os.path.join('..'))
sys.path.insert(0, module_path)

from src.config import cfg
from src.generate import predict_single, encode, predict_single_mod
from src.dataset import get_validation_data
import src.diverse_beam_search as mbs

from onnxruntime import InferenceSession, SessionOptions, ExecutionMode

Setting bashlex grammar using file: /home/jaron/shared/internship-jaron/bashlint/grammar/grammar100.txt
Bashlint grammar set up (148 utilities)



In [2]:
model_path = "output/nl2bash/08-26_18:11:32"
model_name = "gpt2-large"
model = GPT2LMHeadModel.from_pretrained(model_path)
set_cfg(run)
set_cfg('device', 'cuda')
tokenizer = get_tokenizer()
model = get_model(tokenizer, resume=True)


In [19]:
def encode(prompt):
    return f"{cfg('eos')} {cfg('sep1')} {prompt}\n{cfg('sep2')}"


def decode(tokenizer, v):
    text = tokenizer.decode(v, clean_up_tokenization_spaces=False)
    # remove query at the start
    start = text.find(cfg('sep2')) + len(cfg('sep2'))
    text = text[start:]
    # remove possible junk at the end
    end = text.find("\n")
    if end!=-1:
        text = text[:end]
    text = text.strip('\n ')
    return text


def decode_batch(tokenizer, vs):
    return [decode(tokenizer, v) for v in vs]

def tokenize_query(tokenizer, prompt):
    prompt = encode(prompt)
    encoded_prompt = tokenizer.encode(prompt, return_tensors="pt")
    return encoded_prompt.to(device)


def predict_single_div(model, tokenizer, prompt, top=1, max_length=None, beams=None):
    prompt = tokenize_query(tokenizer, prompt)
    
    PreTrainedModel._generate_beam_search = mbs._generate_beam_search
    output = model.generate(
        input_ids=prompt,
        max_length=200,
        num_beams=beams,
        do_sample=False,
        num_return_sequences=top,
        pad_token_id=tokenizer.eos_token_id,
        #fp16 = True
        )
    output = decode_batch(tokenizer, output)
    if len(output) == 1:
        return output[0]
    else:
        return output

In [36]:
importlib.reload(mbs)


<module 'src.diverse_beam_search' from '/home/jaron/shared/internship-jaron/src/diverse_beam_search.py'>

In [37]:
%%time
prompt = 'search all pdf files large than 10GB in the current folder'
q = predict_single_div(model, tokenizer, prompt, beams=5, top=5)

print("RESULTS")
for cm in q:
    print(cm)

DIVERSE BEAM SEARCH 2
RESULTS
find Path -size +Size -print0 | xargs -0 -I {} grep -l -H Regex {}
find Path -size +Size -name Regex -print
find Path -iname Regex -size +Size
find Path -size +Size -print | xargs -I {} grep -l Regex {}
find Path $(find Path -size +Size) -type f -size +Size
CPU times: user 5.38 s, sys: 22.9 ms, total: 5.41 s
Wall time: 1.02 s


In [34]:
%%time
query = 'search all pdf files large than 10GB in the current folder"'
q = predict_single_mod(model, tokenizer, query, beams=5, top=5)
print("RESULTS")
for cm in q[0]:
    print(cm)

RESULTS
find Path -size +Size -iname Regex
find Path -size +Size -iname Regex -print
find Path -size +Size -print0 | xargs -0 -I {} grep -H -n Regex {}
find Path -size +Size -name Regex -print
find Path -type f -size +Size -print0 | xargs -0 -I {} grep -H -n Regex {}
CPU times: user 460 ms, sys: 36.1 ms, total: 496 ms
Wall time: 494 ms


# Evaluation

In [ ]:
def get_template_score(dev_cm, pred_cms, confs):
    PARAMS = {'u1': 1., 'u2':1.}
    scores = [compute_metric(pred_cm, conf, dev_cm, PARAMS)
                for pred_cm, conf in zip(pred_cms, confs)]

    if any(x > 0 for x in scores):
        total_score = max(scores)
    else:
        total_score = sum(scores) / len(scores)
    return total_score

def validate(model, tokenizer, dev_nls, dev_cms, overconfident=False, zero_hack=False):
    outputs = [predict_single_mod(model, tokenizer, dev_nl, top=cfg('val_n')) 
                for dev_nl in dev_nls]
    
    confidences = [x[1] for x in outputs]
    predictions = [x[0] for x in outputs]
    
    if overconfident:
        confidences = [[1. for _ in entry] for entry in confidences]
    
    if zero_hack:
        for entry in confidences:
            entry[-1] = 0

    scores_template = [get_template_score(dev_cm[0], pred_cm, confs) 
                for (pred_cm, confs, dev_cm) in zip(predictions, confidences, dev_cms)]
    print(f"[DEBUG]: TM score {np.mean(scores_template)}")

    scores_blue = [sentence_bleu(dev_cm, pred_cm[0]) 
                for (pred_cm, dev_cm) in zip(predictions, dev_cms)]
    print(f"[DEBUG]: BLUE score {np.mean(scores_blue)}")

    if cfg('val_metric') == 'BLUE':
        scores = scores_blue
    elif cfg('val_metric') == 'template':
        scores = scores_template
    else:
        assert False, f"Unkown validation metric '{metric}'"
    return scores, predictions

In [38]:
dev_nls, dev_cms = get_validation_data()

FileNotFoundError: [Errno 2] No such file or directory: 'data/nl2bash/dev_cm.txt'